In [6]:
import sys
import os
import torch
import librosa
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [9]:
import os
import sys

notebook_dir = r"C:\Users\sansk\Downloads\ai_noise_reduction\Notebooks"
sys.path.append(notebook_dir)
os.chdir(notebook_dir)  # Optional: Set working directory to this path

print("New working directory:", os.getcwd())

New working directory: C:\Users\sansk\Downloads\ai_noise_reduction\Notebooks


In [10]:
from dataset import NoiseReductionDataset
from model import SimpleDenoisingCNN

In [11]:
noisy_path = 'C:\\Users\\sansk\\Downloads\\ai_noise_reduction\\data\\clean_testset_wav'
clean_path = 'C:\\Users\\sansk\\Downloads\\ai_noise_reduction\\data\\noisy_dataset_wav'

dataset = NoiseReductionDataset(noisy_path, clean_path)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)


Found 824 noisy files and 824 clean files.


In [15]:
# Define model
model = SimpleDenoisingCNN()

# Correct path to the model
model_path = os.path.join("..", "model", "Denoising_model.pth")

# Load model weights
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

SimpleDenoisingCNN(
  (encoder): Sequential(
    (0): Conv1d(1, 16, kernel_size=(15,), stride=(1,), padding=(7,))
    (1): ReLU()
    (2): Conv1d(16, 8, kernel_size=(15,), stride=(1,), padding=(7,))
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): Conv1d(8, 16, kernel_size=(15,), stride=(1,), padding=(7,))
    (1): ReLU()
    (2): Conv1d(16, 1, kernel_size=(15,), stride=(1,), padding=(7,))
  )
)

In [16]:
#Evaluation Loop
all_preds = []
all_targets = []

with torch.no_grad():
    for noisy, clean in dataloader:
        # Convert shape: [batch_size, seq_len] → [batch_size, 1, seq_len]
        noisy = noisy.unsqueeze(1).float()
        clean = clean.unsqueeze(1).float()

        output = model(noisy)

        # Flatten for metric calculation
        output_np = output.squeeze().numpy().flatten()
        clean_np = clean.squeeze().numpy().flatten()

        all_preds.extend(output_np)
        all_targets.extend(clean_np)


In [17]:
#Calculate Metrics
all_preds = np.array(all_preds)
all_targets = np.array(all_targets)

# Threshold to create binary mask if needed (e.g., for speech activity detection type task)
threshold = 0.05
binary_preds = (all_preds > threshold).astype(int)
binary_targets = (all_targets > threshold).astype(int)

mse = mean_squared_error(all_targets, all_preds)
precision = precision_score(binary_targets, binary_preds, zero_division=0)
recall = recall_score(binary_targets, binary_preds, zero_division=0)
f1 = f1_score(binary_targets, binary_preds, zero_division=0)

print(f"Mean Squared Error: {mse:.6f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Mean Squared Error: 0.015115
Precision: 0.9860
Recall: 0.2599
F1 Score: 0.4114


In [29]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score

# Example: Reshaping the input to have 1 channel and the length as the other dimensions
X_test = np.random.rand(5, 3)  # Example: 5 samples, each with 3 features

# Reshape it to match the expected input of [batch_size, channels, input_length]
# For this example, reshape to [5, 1, 3] where 5 is batch size, 1 is channel, and 3 is length
X_test_reshaped = X_test.reshape(5, 1, 3)

# Convert to PyTorch tensor
X_test_tensor = torch.tensor(X_test_reshaped, dtype=torch.float32)

# Example: Defining the model (replace with your actual model)
# For demonstration, we're using a simple dummy model
class DummyModel(torch.nn.Module):
    def __init__(self):
        super(DummyModel, self).__init__()
        self.layer = torch.nn.Linear(3, 1)  # 3 features input, 1 output (binary)

    def forward(self, x):
        return torch.sigmoid(self.layer(x))

model = DummyModel()

# Example true labels (y_true in binary format)
y_true = [1, 0, 1, 0, 1]  # True labels (must be in binary format)

# Now pass it to the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)

# Convert the model output to binary predictions (0 or 1)
y_pred_binary = (y_pred > 0.5).float()

# Calculate accuracy by converting y_pred_binary to a numpy array
accuracy = accuracy_score(y_true, y_pred_binary.numpy())
print(f"Accuracy: {accuracy}")


ValueError: Classification metrics can't handle a mix of binary and unknown targets

In [28]:
import torch
from sklearn.metrics import accuracy_score

# Example model output (probabilities)
model_output = torch.tensor([0.7, 0.2, 0.9, 0.4, 0.6])

# Convert model output to binary predictions (0 or 1)
y_pred_binary = (model_output > 0.5).float()

# Example true labels (binary format)
y_true = [1, 0, 1, 0, 1]

# Ensure y_pred_binary is a NumPy array for compatibility with accuracy_score
accuracy = accuracy_score(y_true, y_pred_binary.numpy())
print(f"Accuracy: {accuracy}")


Accuracy: 1.0
